In [8]:
import os
import numpy as np
from tqdm import tqdm 
from PIL import Image

def read_images(path):
    files = os.listdir(path)
    files = [file for file in files if file[-4:] == ".jpg"]
    
    images = []
    for file in tqdm(files):
        try:
            image = Image.open(path + "/" + file)
            image = image.resize((224,224), Image.LANCZOS)
            image = image.convert("RGB")

            image = np.asarray(image)

            images.append(image)
        except OSError:
            pass
        
    return images
    
    
    
    
mona_lisa = read_images("Desktop/Bilder/mona_lisa")
mona_lisa = np.asarray(mona_lisa)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 43.42it/s]


In [9]:
mona_lisa.shape

(50, 224, 224, 3)

In [25]:
import os
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop

In [11]:
#X = preprocess_input(mona_lisa)

In [26]:
vgg16_model = VGG16(include_top = False,input_shape=(224,224,3))
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [27]:
model = Sequential()
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(2, activation="sigmoid"))

#model.compile(optimizer=Adam(learning_rate=0.00001),loss = "categorical_crossentropy", metrics = ["acc"])
#model.summary()

In [28]:
TRAINING_DIR = "Desktop/Bilder/"
training_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size = (224,224),
    class_mode = 'categorical'
)

Found 144 images belonging to 2 classes.


In [29]:
VALIDATION_DIR = "Desktop/Bilder/"
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size = (224,224),
    class_mode = 'categorical'
)

Found 144 images belonging to 2 classes.


In [30]:
model.compile(optimizer=Adam(learning_rate=0.00001),loss = "categorical_crossentropy", metrics = ["acc"])

In [ ]:
model.fit(train_generator, epochs = 25,
    validation_data = validation_generator,
    verbose = 1)

Epoch 1/25
3/5 [=================>............] - ETA: 1:54 - loss: 0.7652 - acc: 0.4625

In [ ]:
labels = ["Michelangelo-ErschaffungAdams", "Michelangelo-Die heilige Familie", "VanGogh-Selbstbildnis"]